In [ ]:
from pyspark.sql.types import FloatType
from pyspark.sql.functions import col
from pyspark.sql import DataFrame
from functools import reduce 

## Connect with azure datalake using OAuth 2.0 with an Azure Active Directory (Azure AD)

In [ ]:
service_credential = dbutils.secrets.get(scope="key_vault_scope",key="service-credential-key")

spark.conf.set("fs.azure.account.auth.type.minedata.dfs.core.windows.net", "OAuth")
spark.conf.set("fs.azure.account.oauth.provider.type.minedata.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
spark.conf.set("fs.azure.account.oauth2.client.id.minedata.dfs.core.windows.net", "fec6b5a7-ad07-45e2-b61b-dc037c92904f")
spark.conf.set("fs.azure.account.oauth2.client.secret.minedata.dfs.core.windows.net", service_credential)
spark.conf.set("fs.azure.account.oauth2.client.endpoint.minedata.dfs.core.windows.net", "https://login.microsoftonline.com/3417d36b-fa61-4b84-b95e-8414a4e5753f/oauth2/token")

## Read minining data from two machines

In [ ]:
# Read machine one data
machine_one_data = spark.read.option("header",True).csv("abfss://miningdata@minedata.dfs.core.windows.net/plant_one/MiningProcess_Flotation_Plant_Database_1.csv")

# Read machine two data
machine_two_data = spark.read.option("header",True).csv("abfss://miningdata@minedata.dfs.core.windows.net/plant_two/MiningProcess_Flotation_Plant_Database_2.csv")


## Check the schema of both machine data

In [ ]:
machine_one_data.printSchema()

root
-- _c0: string (nullable = true)
-- date: string (nullable = true)
-- % Iron Feed: string (nullable = true)
-- % Silica Feed: string (nullable = true)
-- Starch Flow: string (nullable = true)
-- Amina Flow: string (nullable = true)
-- Ore Pulp Flow: string (nullable = true)
-- Ore Pulp pH: string (nullable = true)
-- Ore Pulp Density: string (nullable = true)
-- Flotation Column 01 Air Flow: string (nullable = true)
-- Flotation Column 02 Air Flow: string (nullable = true)
-- Flotation Column 03 Air Flow: string (nullable = true)
-- Flotation Column 04 Air Flow: string (nullable = true)
-- Flotation Column 05 Air Flow: string (nullable = true)
-- Flotation Column 06 Air Flow: string (nullable = true)
-- Flotation Column 07 Air Flow: string (nullable = true)
-- Flotation Column 01 Level: string (nullable = true)
-- Flotation Column 02 Level: string (nullable = true)
-- Flotation Column 03 Level: string (nullable = true)
-- Flotation Column 04 Level: string (nullable = true)
-- Flotation Column 05 Level: string (nullable = true)
-- Flotation Column 06 Level: string (nullable = true)
-- Flotation Column 07 Level: string (nullable = true)
-- % Iron Concentrate: string (nullable = true)
-- % Silica Concentrate: string (nullable = true)

In [ ]:
machine_two_data.printSchema()

root
-- _c0: string (nullable = true)
-- date: string (nullable = true)
-- % Iron Feed: string (nullable = true)
-- % Silica Feed: string (nullable = true)
-- Starch Flow: string (nullable = true)
-- Amina Flow: string (nullable = true)
-- Ore Pulp Flow: string (nullable = true)
-- Ore Pulp pH: string (nullable = true)
-- Ore Pulp Density: string (nullable = true)
-- Flotation Column 01 Air Flow: string (nullable = true)
-- Flotation Column 02 Air Flow: string (nullable = true)
-- Flotation Column 03 Air Flow: string (nullable = true)
-- Flotation Column 04 Air Flow: string (nullable = true)
-- Flotation Column 05 Air Flow: string (nullable = true)
-- Flotation Column 06 Air Flow: string (nullable = true)
-- Flotation Column 07 Air Flow: string (nullable = true)
-- Flotation Column 01 Level: string (nullable = true)
-- Flotation Column 02 Level: string (nullable = true)
-- Flotation Column 03 Level: string (nullable = true)
-- Flotation Column 04 Level: string (nullable = true)
-- Flotation Column 05 Level: string (nullable = true)
-- Flotation Column 06 Level: string (nullable = true)
-- Flotation Column 07 Level: string (nullable = true)
-- % Iron Concentrate: string (nullable = true)
-- % Silica Concentrate: string (nullable = true)

## Data exploaration

- Drop unwanted columns
- Convert string into float
- Merge both plant data and load into the datalake

In [ ]:
# Check the existing number of columns
print(f"Number of machine one data features: {len(machine_one_data.columns)}")
print(f"Number of machine two data features: {len(machine_two_data.columns)}")

Number of machine one data features: 25
Number of machine two data features: 25

In [ ]:
machine_one_data.show(1)

+---+-------------------+-----------+-------------+-----------+----------+-------------+-----------+----------------+----------------------------+----------------------------+----------------------------+----------------------------+----------------------------+----------------------------+----------------------------+-------------------------+-------------------------+-------------------------+-------------------------+-------------------------+-------------------------+-------------------------+------------------+--------------------+
_c0| date|% Iron Feed|% Silica Feed|Starch Flow|Amina Flow|Ore Pulp Flow|Ore Pulp pH|Ore Pulp Density|Flotation Column 01 Air Flow|Flotation Column 02 Air Flow|Flotation Column 03 Air Flow|Flotation Column 04 Air Flow|Flotation Column 05 Air Flow|Flotation Column 06 Air Flow|Flotation Column 07 Air Flow|Flotation Column 01 Level|Flotation Column 02 Level|Flotation Column 03 Level|Flotation Column 04 Level|Flotation Column 05 Level|Flotation Column 06 Level|Flotation Column 07 Level|% Iron Concentrate|% Silica Concentrate|
+---+-------------------+-----------+-------------+-----------+----------+-------------+-----------+----------------+----------------------------+----------------------------+----------------------------+----------------------------+----------------------------+----------------------------+----------------------------+-------------------------+-------------------------+-------------------------+-------------------------+-------------------------+-------------------------+-------------------------+------------------+--------------------+
 0|2017-03-10 01:00:00| 55,2| 16,98| 3019,53| 557,434| 395,713| 10,0664| 1,74| 249,214| 253,235| 250,576| 295,096| 306,4| 250,225| 250,884| 457,396| 432,962| 424,954| 443,558| 502,255| 446,37| 523,344| 66,91| 1,31|
+---+-------------------+-----------+-------------+-----------+----------+-------------+-----------+----------------+----------------------------+----------------------------+----------------------------+----------------------------+----------------------------+----------------------------+----------------------------+-------------------------+-------------------------+-------------------------+-------------------------+-------------------------+-------------------------+-------------------------+------------------+--------------------+
only showing top 1 row

### Drop unwanted columns

In [ ]:
# Create column name list
col_list = machine_one_data.columns
print(col_list)
print(f"Length is :{len(col_list)}")
# Drop unwanted column name from the list
col_list = col_list[3:23]
print(col_list)

['_c0', 'date', '% Iron Feed', '% Silica Feed', 'Starch Flow', 'Amina Flow', 'Ore Pulp Flow', 'Ore Pulp pH', 'Ore Pulp Density', 'Flotation Column 01 Air Flow', 'Flotation Column 02 Air Flow', 'Flotation Column 03 Air Flow', 'Flotation Column 04 Air Flow', 'Flotation Column 05 Air Flow', 'Flotation Column 06 Air Flow', 'Flotation Column 07 Air Flow', 'Flotation Column 01 Level', 'Flotation Column 02 Level', 'Flotation Column 03 Level', 'Flotation Column 04 Level', 'Flotation Column 05 Level', 'Flotation Column 06 Level', 'Flotation Column 07 Level', '% Iron Concentrate', '% Silica Concentrate']
Length is :25
['% Silica Feed', 'Starch Flow', 'Amina Flow', 'Ore Pulp Flow', 'Ore Pulp pH', 'Ore Pulp Density', 'Flotation Column 01 Air Flow', 'Flotation Column 02 Air Flow', 'Flotation Column 03 Air Flow', 'Flotation Column 04 Air Flow', 'Flotation Column 05 Air Flow', 'Flotation Column 06 Air Flow', 'Flotation Column 07 Air Flow', 'Flotation Column 01 Level', 'Flotation Column 02 Level', 'Flotation Column 03 Level', 'Flotation Column 04 Level', 'Flotation Column 05 Level', 'Flotation Column 06 Level', 'Flotation Column 07 Level']

In [ ]:
# Select required column
machine_one_data = machine_one_data.select(*col_list)
machine_two_data = machine_two_data.select(*col_list)

In [ ]:
print(f"Number of machine one data features: {len(machine_one_data.columns)}")
print(f"Number of machine two data features: {len(machine_two_data.columns)}")

Number of machine one data features: 20
Number of machine two data features: 20

### Convert sting data to float

In [ ]:
machine_one_data.show(1)

+-------------+-----------+----------+-------------+-----------+----------------+----------------------------+----------------------------+----------------------------+----------------------------+----------------------------+----------------------------+----------------------------+-------------------------+-------------------------+-------------------------+-------------------------+-------------------------+-------------------------+-------------------------+
% Silica Feed|Starch Flow|Amina Flow|Ore Pulp Flow|Ore Pulp pH|Ore Pulp Density|Flotation Column 01 Air Flow|Flotation Column 02 Air Flow|Flotation Column 03 Air Flow|Flotation Column 04 Air Flow|Flotation Column 05 Air Flow|Flotation Column 06 Air Flow|Flotation Column 07 Air Flow|Flotation Column 01 Level|Flotation Column 02 Level|Flotation Column 03 Level|Flotation Column 04 Level|Flotation Column 05 Level|Flotation Column 06 Level|Flotation Column 07 Level|
+-------------+-----------+----------+-------------+-----------+----------------+----------------------------+----------------------------+----------------------------+----------------------------+----------------------------+----------------------------+----------------------------+-------------------------+-------------------------+-------------------------+-------------------------+-------------------------+-------------------------+-------------------------+
 16,98| 3019,53| 557,434| 395,713| 10,0664| 1,74| 249,214| 253,235| 250,576| 295,096| 306,4| 250,225| 250,884| 457,396| 432,962| 424,954| 443,558| 502,255| 446,37| 523,344|
+-------------+-----------+----------+-------------+-----------+----------------+----------------------------+----------------------------+----------------------------+----------------------------+----------------------------+----------------------------+----------------------------+-------------------------+-------------------------+-------------------------+-------------------------+-------------------------+-------------------------+-------------------------+
only showing top 1 row

In [ ]:
# Define user defined function
def convert_float(str):
    str = str.replace(",", ".")
    str = str.strip()
    return float(str)

convert_float_udf = udf(lambda z: convert_float(z), FloatType())

In [ ]:
for col_name in machine_one_data.columns:
    machine_one_data = machine_one_data.withColumn(col_name, convert_float_udf(col(col_name)))
    
for col_name in machine_two_data.columns:
    machine_two_data = machine_two_data.withColumn(col_name, convert_float_udf(col(col_name)))

In [ ]:
machine_two_data.show(1)

+-------------+-----------+----------+-------------+-----------+----------------+----------------------------+----------------------------+----------------------------+----------------------------+----------------------------+----------------------------+----------------------------+-------------------------+-------------------------+-------------------------+-------------------------+-------------------------+-------------------------+-------------------------+
% Silica Feed|Starch Flow|Amina Flow|Ore Pulp Flow|Ore Pulp pH|Ore Pulp Density|Flotation Column 01 Air Flow|Flotation Column 02 Air Flow|Flotation Column 03 Air Flow|Flotation Column 04 Air Flow|Flotation Column 05 Air Flow|Flotation Column 06 Air Flow|Flotation Column 07 Air Flow|Flotation Column 01 Level|Flotation Column 02 Level|Flotation Column 03 Level|Flotation Column 04 Level|Flotation Column 05 Level|Flotation Column 06 Level|Flotation Column 07 Level|
+-------------+-----------+----------+-------------+-----------+----------------+----------------------------+----------------------------+----------------------------+----------------------------+----------------------------+----------------------------+----------------------------+-------------------------+-------------------------+-------------------------+-------------------------+-------------------------+-------------------------+-------------------------+
 6.65| 3035.16| 404.663| 396.237| 10.0073| 1.6003| 300.638| 294.625| 300.354| 298.68| 301.873| 300.01| 299.714| 398.079| 414.498| 398.394| 400.196| 380.584| 392.546| 407.305|
+-------------+-----------+----------+-------------+-----------+----------------+----------------------------+----------------------------+----------------------------+----------------------------+----------------------------+----------------------------+----------------------------+-------------------------+-------------------------+-------------------------+-------------------------+-------------------------+-------------------------+-------------------------+
only showing top 1 row

### Merge both plant data and load into the datalake

In [ ]:
df = reduce(DataFrame.unionAll, [machine_one_data, machine_two_data])
print(f"Number of machine one data features: {machine_one_data.count()}")
print(f"Number of machine two data features: {machine_two_data.count()}")
print(f"Total number of machine data:{df.count()}")

Number of machine one data features: 368727
Number of machine two data features: 368726
Total number of machine data:737453

In [ ]:
df.write.option("header",True).csv("abfss://miningdata@minedata.dfs.core.windows.net/processed.csv")